In [ ]:
import chess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten

In [ ]:
# Defining Data preparation functions
def one_hot_encode_piece(piece):
    pieces = list('rnbqkpRNBQKP.')
    arr = np.zeros(len(pieces))
    piece_to_index = {p: i for i, p in enumerate(pieces)}
    index = piece_to_index[piece]
    arr[index] = 1
    return arr

def encode_board(board):
    # first lets turn the board into a string
    board_str = str(board)
    # then lets remove all the spaces
    board_str = board_str.replace(' ', '')
    board_list = []
    for row in board_str.split('\n'):
        row_list = []
        for piece in row:
            row_list.append(one_hot_encode_piece(piece))
        board_list.append(row_list)
    return np.array(board_list)

def encode_fen_string(fen_str):
    board = chess.Board(fen=fen_str)
    return encode_board(board)

In [ ]:
# Pulling in training data using Pandas
train_df = pd.read_csv('datasets/train.csv', index_col='id')

# We'll only use the first 10000 examples so things run fast,
# but you'll get better performance if you remove this line
train_df = train_df[:25000]

# We'll also grab the last 1000 examples as a validation set
val_df = train_df[-1000:]
train_df.head()

def encode_fen_string(fen_str):
    board = chess.Board(fen=fen_str)
    return encode_board(board)

# We'll stack all our encoded boards into a single numpy array
X_train = np.stack(train_df['board'].apply(encode_fen_string))
y_train = train_df['black_score']


X_val = np.stack(val_df['board'].apply(encode_fen_string))
y_val = val_df['black_score']

In [ ]:
# Creating our prediction model
# With the Keras Sequential model we can stack neural network layers like Dense, Flatten together
model = Sequential([
    Flatten(),
    Dense(2048, activation='relu'),
    Dense(1),
])

model.compile(
    optimizer='rmsprop',
    loss='mean_squared_error')

In [ ]:
# Solve the no attribute 'DistributedDatasetInterface' problem
from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset

# To test things out, let's train for n epochs and see how our model is doing
history = model.fit(
    X_train,
    y_train,
    epochs=40,
    validation_data=(X_val, y_val))

In [ ]:
# Plotting our latest training results
plt.style.use('ggplot')
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title('Loss During Training')
plt.show()

In [ ]:
# Implementing our model as a function
def play_nn(fen, show_move_evaluations=False):
    # We can create a python-chess board instance from the FEN string like this:
    board = chess.Board(fen=fen)

    # And then evaluate all legal moves
    moves = []
    input_vectors = []
    for move in board.legal_moves:
        # For each move, we'll make a copy of the board and try that move out
        candidate_board = board.copy()
        candidate_board.push(move)
        moves.append(move)
        input_vectors.append(encode_board(str(candidate_board)).astype(np.int32).flatten())

    input_vectors = np.stack(input_vectors)
    # This is where our model gets to shine! It tells us how good the resultant score board is for black:
    scores = model.predict(input_vectors, verbose=0)
    # argmax gives us the index of the highest scoring move
    if board.turn == chess.BLACK:
        index_of_best_move = np.argmax(scores)
    else:
        # If we're playing as white, we want black's score to be as small as possible, so we take argmax of the negative of our array
        index_of_best_move = np.argmax(-scores)

    if show_move_evaluations:
        print(zip(moves, scores))

    best_move = moves[index_of_best_move]

    # Now we turn our move into a string, return it and call it a day!
    return str(best_move)

In [ ]:
# Now we'll import our test set, and make some final predictions!

test_df = pd.read_csv('datasets/test.csv')

test_df.head()

In [ ]:
# Making all of our predictions happens in this one line!
# We're basically saying "run play_nn on all the boards in the test_df, and then keep the results as best_move"
# Because this invovles running our model a _ton_ this step will take a while.

test_df['best_move'] = test_df['board'].apply(play_nn)

In [ ]:
test_df['best_move']

In [ ]:
# Let's make sure our submission looks like the sample submission
submission = test_df[['id', 'best_move']]
print(submission.head())

sample_submission = pd.read_csv('datasets/sample_submission.csv', index_col='id')
print(sample_submission.head())

In [ ]:
submission.to_csv('submission.csv', index=False)